In [ ]:
import re
from bs4 import BeautifulSoup
import pandas as pd

def review_to_wordlist(review):
    review_text = BeautifulSoup(review).get_text()
    
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    
    words = review_text.lower().split()
    return words

train = pd.read_csv('/Users/Hanxiaoyang/IMDB_sentiment_analysis_data/labeledTrainData.tsv', header=0, delimiter="\t", quoting=3)
test = pd.read_csv('/Users/Hanxiaoyang/IMDB_sentiment_analysis_data/testData.tsv', header=0, delimiter="\t", quoting=3 )


y_train = train['sentiment']
train_data =[]

for i in range(0,len(train['review'])):
    train_data.append(" ".join(review_to_wordlist(train['review'][i])))

test_data =[]

for i in range(0,len(train['review'])):
    test_data.append(" ".join(review_to_wordlist(test['review'][i])))
    
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf

tfv = tfidf(min_df=3,max_features=None,strip_accents='unicode',analyzer='word',
            token_pattern=r'\w{1,}', ngram_range=(1,2),use_idf=1,
            smooth_idf=1,sublinear_tf=1,stop_words='english')

X_all = train_data + test_data
len_train = len(train_data)

tfv.fit(X_all)
X_all = tfv.transform(X_all)

X_train = X_all[:len_train]
X_test = X_all[len_train:]

from sklearn.naive_bayes import MultinomialNB as MNB

model_NB = MNB()
model_NB.fit(X, y_train) #特征数据直接灌进来
MNB(alpha=1.0, class_prior=None, fit_prior=True)

from sklearn.cross_validation import cross_val_score
import numpy as np

print ("多项式贝叶斯分类器20折交叉验证得分: ", np.mean(cross_val_score(model_NB, X, y_train, cv=20, scoring='roc_auc')))
# 多项式贝叶斯分类器20折交叉验证得分: 0.950837239



from sklearn.linear_model import LogisticRegression as LR
from sklearn.grid_search import GridSearchCV

# 设定grid search的参数
grid_values = {'C':[30]}
# 设定打分为roc_auc
model_LR = GridSearchCV(LR(penalty = 'L2', dual = True, random_state = 0), grid_values, scoring = 'roc_auc', cv = 20)
# 数据灌进来
model_LR.fit(X,y_train)
# 20折交叉验证，开始漫长的等待...
GridSearchCV(cv=20, estimator=LogisticRegression(C=1.0, class_weight=None, dual=True,
                                                 fit_intercept=True, intercept_scaling=1, penalty='L2', random_state=0, tol=0.0001),
             fit_params={}, iid=True, loss_func=None, n_jobs=1,
             param_grid={'C': [30]}, pre_dispatch='2*n_jobs', refit=True,
             score_func=None, scoring='roc_auc', verbose=0)
#输出结果
print (model_LR.grid_scores_)












































